In [1]:
import sys
from pathlib import Path

# Add the project directory to the Python path
project_dir = Path.cwd().parent
sys.path.append(str(project_dir))

from data import Data
from cnn import CNN
import pickle

# Load Dataset

- Preprocess
- Build splits (Training/Validation/Testing):
  - 80/20/00 -> For the hyperparameter search
  - 80/10/10
  - 40/20/40
  - 10/10/80

In [2]:
dataset0 = Data('caltech101_silhouettes_28.mat', train_split=80, val_split=20, test_split=0)
dataset1 = Data('caltech101_silhouettes_28.mat', train_split=80, val_split=10, test_split=10)
dataset2 = Data('caltech101_silhouettes_28.mat', train_split=40, val_split=20, test_split=40)
dataset3 = Data('caltech101_silhouettes_28.mat', train_split=10, val_split=10, test_split=80)

# Hyperparameter Search
Previous to the study of CNN configurations

- OL Activation function
- CFL Cost Function
- Dense Layer Size
- Learning rates

In [ ]:
import itertools
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

output_activations = ['softmax', 'sigmoid']
cost_functions = ['categorical_crossentropy', 'mean_squared_error']
learning_rates = [0.01, 0.1]
dense_layer_sizes = [64, 128]
max_epochs = 20

def train_model_hyperparmeter_search(params):
    try:
        oa, cf, dls, lr = params
        cnn = CNN(output_layer_activation=oa, filter_sizes=[64, 64], dense_layer_size=dls, hidden_layer_activation='tanh')
        
        print(f"Training with parameters: OA={oa}, CF={cf}, DLS={dls}, LR={lr}")
        
        history = cnn.fit(
            dataset0,
            cost_function=cf,
            max_epochs=max_epochs,
            learning_rate=lr
        )
        
        val_acc = history.history['val_accuracy'][-1]
        return (oa, cf, dls, lr, val_acc, history)
    except Exception as e:
        print(f"Error processing parameters {params}:")
        print(traceback.format_exc())
        return None

# Generate all parameter combinations
param_combinations = list(itertools.product(
    output_activations, 
    cost_functions, 
    dense_layer_sizes,
    learning_rates
))

# Use ThreadPoolExecutor to parallelize
hyperparameter_search_results = []

with ThreadPoolExecutor() as executor:
    # Submit all tasks
    futures = [executor.submit(train_model_hyperparmeter_search, params) for params in param_combinations]
    
    # Collect results as they complete
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            hyperparameter_search_results.append(result)

# Sort and display results
hyperparameter_search_results = sorted(hyperparameter_search_results, key=lambda x: x[-2], reverse=True)
for result in hyperparameter_search_results:
    print(f"Params: {result[:-2]}, Validation Accuracy: {result[-2]:.4f}")

best_hyperparameters = hyperparameter_search_results[0][:-2]

Training with parameters: OA=softmax, CF=categorical_crossentropy, DLS=64, LR=0.01
Training with parameters: OA=softmax, CF=categorical_crossentropy, DLS=64, LR=0.1
Training with parameters: OA=softmax, CF=categorical_crossentropy, DLS=128, LR=0.01
Training with parameters: OA=softmax, CF=categorical_crossentropy, DLS=128, LR=0.1
Training with parameters: OA=softmax, CF=mean_squared_error, DLS=64, LR=0.01
Training with parameters: OA=softmax, CF=mean_squared_error, DLS=64, LR=0.1
Training with parameters: OA=softmax, CF=mean_squared_error, DLS=128, LR=0.01
Training with parameters: OA=softmax, CF=mean_squared_error, DLS=128, LR=0.1
Training with parameters: OA=sigmoid, CF=categorical_crossentropy, DLS=64, LR=0.01
Training with parameters: OA=sigmoid, CF=categorical_crossentropy, DLS=64, LR=0.1
Training with parameters: OA=sigmoid, CF=categorical_crossentropy, DLS=128, LR=0.01
Training with parameters: OA=sigmoid, CF=categorical_crossentropy, DLS=128, LR=0.1
Training with parameters: OA

In [4]:
# Save the list to a pickle file
with open('hyperparameter_search_results.pkl', 'wb') as file:
    pickle.dump(hyperparameter_search_results, file)

# CNN Results

Loop over configurations:
1. Architecture:
  - 1 Block: 128 Filter Size
  - 3 Blocks: 32, 64 and 128 Filter Sizes
2. Activations:
  - Sigmoid
  - ReLU
3. Dataset Splits.

In [ ]:
import itertools
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

oa = best_hyperparameters[0]
cf =  best_hyperparameters[1]
dls =  best_hyperparameters[2]
lr =  best_hyperparameters[3]

datasets = [dataset1, dataset2, dataset3]
filter_sizes = [[128], [32, 64, 128]]
hidden_activations = ['sigmoid', 'relu']

def train_model_configuration_search(params):
    try:
        dataset, fs, nhl = params
        cnn = CNN(output_layer_activation=oa, filter_sizes=fs, dense_layer_size=dls, hidden_layer_activation=nhl)
        
        splits = dataset.splits
        
        print(f"Training with parameters: FS={fs}, NHL={nhl}, Splits={splits}")
        
        history = cnn.fit(
            dataset,
            cost_function=cf,
            max_epochs=max_epochs,
            learning_rate=lr
        )
        
        val_acc = max(history.history['val_accuracy'])

        test_acc = cnn.evaluate(dataset)

        return (fs, nhl, splits, val_acc, test_acc[1], history)
    except Exception as e:
        print(f"Error processing parameters {params} with dataset {splits}:")
        print(traceback.format_exc())
        return None

# Generate all parameter combinations
param_combinations = list(itertools.product(
    datasets,
    filter_sizes,
    hidden_activations
))

# Use ThreadPoolExecutor to parallelize
configuration_search_results = []
with ThreadPoolExecutor() as executor:
    # Submit all tasks
    futures = [executor.submit(train_model_configuration_search, params) for params in param_combinations]
    
    # Collect results as they complete
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            configuration_search_results.append(result)

# Sort and display results
configuration_search_results = sorted(configuration_search_results, key=lambda x: x[-2], reverse=True)
for result in configuration_search_results:
    print(f"Params: {result[:-3]}, Validation Accuracy: {result[-3]:.4f}, Testing Accuracy: {result[-2]:.4f}")

Training with parameters: FS=[128], NHL=sigmoid, Splits=(80, 10, 10)
Training with parameters: FS=[128], NHL=relu, Splits=(80, 10, 10)
Training with parameters: FS=[32, 64, 128], NHL=sigmoid, Splits=(80, 10, 10)
Training with parameters: FS=[32, 64, 128], NHL=relu, Splits=(80, 10, 10)
Training with parameters: FS=[128], NHL=sigmoid, Splits=(40, 20, 40)
Training with parameters: FS=[128], NHL=relu, Splits=(40, 20, 40)
Training with parameters: FS=[32, 64, 128], NHL=sigmoid, Splits=(40, 20, 40)
Training with parameters: FS=[32, 64, 128], NHL=relu, Splits=(40, 20, 40)
Training with parameters: FS=[128], NHL=sigmoid, Splits=(10, 10, 80)
Training with parameters: FS=[128], NHL=relu, Splits=(10, 10, 80)
Training with parameters: FS=[32, 64, 128], NHL=sigmoid, Splits=(10, 10, 80)
Training with parameters: FS=[32, 64, 128], NHL=relu, Splits=(10, 10, 80)
217/217 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.5014 - loss: 4.9274
217/217 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0971 - los

In [6]:
# Save the list to a pickle file
with open('configuration_search_results.pkl', 'wb') as file:
    pickle.dump(configuration_search_results, file)

# CNN Analysis

In [7]:
with open('hyperparameter_search_results.pkl', 'rb') as file:
    hyperparameter_search_results = pickle.load(file)

with open('configuration_search_results.pkl', 'rb') as file:
    configuration_search_results = pickle.load(file)

Plots and tables comparing the configurations.

Tables with Validation Accuracies:
- Hyperparameter search
- Configuration search

Plots:
- Hyperparameter Heatmaps
- Best run Train-Validation accuracy plot (need to save all of them during the search)

# Tasks

1) Description of the runs with the different configurations that you have performed. -> Sheena
2) Explain how you have selected the rest of parameters. -> Sheena
3) Those tables that you consider necessary to describe the results obtained for the different network configurations. Explain and reason the results presented in the tables.
    - Tables -> Sheena
    - Heatmaps -> Bruno
    - Best run Train-Validation Accuracy plot -> Sheena
4) Your own conclusions with respect the results obtained. -> Bruno